In [ ]:
#!/bin/bash

cd /well/win/users/hsv459/agemapper

module purge
module load Python/3.7.4-GCCcore-8.3.0
# module load CUDA/10.1.243-GCC-8.3.0
# module load cudnn/7.6.5.32-10.1
# module add fsl/5.0.11

# VISIBLE_CUDA_DEVICES=0,1

# determine Ivybridge or Skylake compatibility on this node
CPU_ARCHITECTURE=$(/apps/misc/utils/bin/get-cpu-software-architecture.py)
# CPU_ARCHITECTURE=general

# Error handling
if [[ ! $? == 0 ]]; then
  echo "Fatal error: Please send the following information to the BMRC team: Could not determine CPU software architecture on $(hostname)"
  exit 1
fi

# Activate the ivybridge or skylake version of your python virtual environment
source /well/win/users/hsv459/python/functionmapper-${CPU_ARCHITECTURE}/bin/activate
# source /well/win/users/hsv459/python/functionmapper-skylake/bin/activate

# continue to use your python venv as normal
   
ipython



In [ ]:
import os
import pickle
import numpy as np
import configparser
import pandas as pd
from fsl.data.image import Image
from fsl.utils.image.resample import resampleToPixdims
from fsl.utils.image.roi import roi
from sklearn.model_selection import train_test_split
import fsl.wrappers.wrapperutils as wutils

In [3]:
# data_directory = "/well/win-biobank/projects/imaging/data/data3/subjectsAll/"
# problematic_data_file = "datasets/problematic_data.txt"

# interesting_data_paths = [
"T1/T1_brain_to_MNI.nii.gz",
# "T1/T1_brain_to_MNI_linear.nii.gz",
"dMRI/autoptx_preproc/tractsNormSummed.nii.gz",
# "T1/T1_vbm/T1_GM_to_template_GM_mod.nii.gz",
"T2_FLAIR/T2_FLAIR_brain_to_MNI.nii.gz",
"SWI/T2star_to_MNI.nii.gz",
    
"fMRI/rfMRI_25.dr/dr_stage2.nii.gz",
"fMRI/tfMRI.feat/reg_standard/stats/zstat1.nii.gz",
# "fMRI/tfMRI.feat/reg_standard/stats/zstat2.nii.gz",
# "fMRI/tfMRI.feat/reg_standard/stats/zstat5.nii.gz",
    
# "fMRI/tfMRI.feat/reg_standard/stats/cope1.nii.gz",
# "fMRI/tfMRI.feat/reg_standard/stats/cope2.nii.gz",
# "fMRI/tfMRI.feat/reg_standard/stats/cope5.nii.gz",
    
# "dMRI/TBSS/stats/all_FA_skeletonised.nii.gz",
# "dMRI/TBSS/stats/all_ICVF_skeletonised.nii.gz",
# "dMRI/TBSS/stats/all_ISOVF_skeletonised.nii.gz",
# "dMRI/TBSS/stats/all_L1_skeletonised.nii.gz",
# "dMRI/TBSS/stats/all_L2_skeletonised.nii.gz",
# "dMRI/TBSS/stats/all_L3_skeletonised.nii.gz",
# "dMRI/TBSS/stats/all_MD_skeletonised.nii.gz",
# "dMRI/TBSS/stats/all_MO_skeletonised.nii.gz",
# "dMRI/TBSS/stats/all_OD_skeletonised.nii.gz",
    
"dMRI/TBSS/stats/all_FA.nii.gz",
# "dMRI/TBSS/stats/all_ICVF.nii.gz",
# "dMRI/TBSS/stats/all_ISOVF.nii.gz",
# "dMRI/TBSS/stats/all_L1.nii.gz",
# "dMRI/TBSS/stats/all_L2.nii.gz",
# "dMRI/TBSS/stats/all_L3.nii.gz",
"dMRI/TBSS/stats/all_MD.nii.gz",
# "dMRI/TBSS/stats/all_MO.nii.gz",
# "dMRI/TBSS/stats/all_OD.nii.gz",

# "T1/transforms/T1_to_MNI_warp_jac.nii.gz",
# "T2_FLAIR/lesions/final_mask_to_MNI.nii.gz",
# ]

In [6]:
for path in interesting_data_paths:
    print("scp -r hsv459@rescomp1.well.ox.ac.uk:"+data_directory+"21946586/"+path+' ./data_examples/ ;' )

scp -r hsv459@rescomp1.well.ox.ac.uk:/well/win-biobank/projects/imaging/data/data3/subjectsAll/21946586/T1/T1_brain_to_MNI.nii.gz ./data_examples/ ;
scp -r hsv459@rescomp1.well.ox.ac.uk:/well/win-biobank/projects/imaging/data/data3/subjectsAll/21946586/T1/T1_brain_to_MNI_linear.nii.gz ./data_examples/ ;
scp -r hsv459@rescomp1.well.ox.ac.uk:/well/win-biobank/projects/imaging/data/data3/subjectsAll/21946586/dMRI/autoptx_preproc/tractsNormSummed.nii.gz ./data_examples/ ;
scp -r hsv459@rescomp1.well.ox.ac.uk:/well/win-biobank/projects/imaging/data/data3/subjectsAll/21946586/T1/T1_vbm/T1_GM_to_template_GM_mod.nii.gz ./data_examples/ ;
scp -r hsv459@rescomp1.well.ox.ac.uk:/well/win-biobank/projects/imaging/data/data3/subjectsAll/21946586/T2_FLAIR/T2_FLAIR_brain_to_MNI.nii.gz ./data_examples/ ;
scp -r hsv459@rescomp1.well.ox.ac.uk:/well/win-biobank/projects/imaging/data/data3/subjectsAll/21946586/SWI/T2star_to_MNI.nii.gz ./data_examples/ ;
scp -r hsv459@rescomp1.well.ox.ac.uk:/well/win-bioban

In [ ]:
interesting_data_paths = ["fMRI/rfMRI_25.dr/dr_stage2.nii.gz",]
data_directory = "/well/win-biobank/projects/imaging/data/data3/subjectsAll/"
problematic_data_file = "datasets/problematic_data.txt"

subDirectoryList = []
number_of_subjects = 0
subject_number = len(os.listdir(os.path.join(os.path.expanduser("~"), data_directory)))
problematic_data = open(problematic_data_file,'r').read().split('\n')

index = 0
directory_list = os.listdir(data_directory)
for directory in directory_list:
    if directory in problematic_data:
        pass
    else:
        if os.path.isdir(os.path.join(data_directory, directory)):
            flag = True
            for modality_path in interesting_data_paths:
                if os.path.exists(os.path.join(data_directory, directory, modality_path)) == False:
                    flag = False
            if flag == True:
                filename = data_directory+directory
                if os.access(filename, os.R_OK):
                    subDirectoryList.append(directory)
                    number_of_subjects += 1
    index +=1
    print("\r Processed {:.3f}%: {}/{} directories".format(index/len(directory_list) * 100.0, index+1, len(directory_list)), end='')
    
print("Number of usable subjects: {}/{}".format(number_of_subjects, len(directory_list)))
print("Length of subDirectoryList: ", len(subDirectoryList))

with open('/path/to/filename.txt', mode='wt', encoding='utf-8') as myfile:
    myfile.write('\n'.join(lines))

def stats_calc(array):

    min_val = np.min(array)
    max_val = np.max(array)
    mean_val = np.mean(array)
    med_val = np.median(array)
    std_val = np.std(array)
    perc1 = np.percentile(array, 1)
    perc25 = np.percentile(array, 25)
    perc75 = np.percentile(array, 75)
    perc99 = np.percentile(array, 99)
    
    norm99p = np.percentile(array / mean_val, 99)

    return min_val, max_val, mean_val, med_val, std_val, perc1, perc25, perc75, perc99, norm99p

In [ ]:

data_directory = "/well/win-biobank/projects/imaging/data/data3/subjectsAll/"
train_inputs = interesting_data_paths[0]
len_subjects = len(subDirectoryList)

print(train_inputs)

data_dict = {}

dictionary_labels = ['min', 'max', 'mean', 'med', 'std', '1p', '25p', '75p', '99p', 'norm99p']

index = 0

for subject in subDirectoryList:

    index += 1

    # ------------------ 

    path = os.path.join(os.path.expanduser("~"), data_directory, subject, train_inputs)
    volume = Image(path).data

    min_val, max_val, mean_val, med_val, std_val, perc1, perc25, perc75, perc99, norm99p = stats_calc(volume)

    data_dict[subject] = [min_val, max_val, mean_val, med_val, std_val, perc1, perc25, perc75, perc99, norm99p]

    del path,volume
    del min_val, max_val, mean_val, med_val, std_val, perc1, perc25, perc75, perc99, norm99p

    print("\r Processed {:.3f}%: {}/{} subjects".format(index/len_subjects * 100.0, index, len_subjects), end='')

    # ------------------
    
if train_inputs == "T1/T1_brain_to_MNI.nii.gz":
    save_path = 'data_analysis/T1_brain_to_MNI.pkl'
    
elif train_inputs == "T1/T1_brain_to_MNI_linear.nii.gz":
    save_path = "data_analysis/T1_brain_to_MNI_linear.pkl"
    
elif train_inputs == "dMRI/autoptx_preproc/tractsNormSummed.nii.gz":
    save_path = "data_analysis/tractsNormSummed.pkl"
    
elif train_inputs == "T1/T1_vbm/T1_GM_to_template_GM_mod.nii.gz":
    save_path = "data_analysis/T1_GM_to_template_GM_mod.pkl"
    
elif train_inputs == "T2_FLAIR/T2_FLAIR_brain_to_MNI.nii.gz":
    save_path = "data_analysis/T2_FLAIR_brain_to_MNI.pkl"
    
elif train_inputs == "SWI/T2star_to_MNI.nii.gz":
    save_path = "data_analysis/T2star_to_MNI.pkl"

elif train_inputs == "fMRI/tfMRI.feat/reg_standard/stats/zstat1.nii.gz":
    save_path = "data_analysis/zstat1.pkl"
    
elif train_inputs == "fMRI/tfMRI.feat/reg_standard/stats/zstat2.nii.gz":
    save_path = "data_analysis/zstat2.pkl"
    
elif train_inputs == "fMRI/tfMRI.feat/reg_standard/stats/zstat5.nii.gz":
    save_path = "data_analysis/zstat5.pkl"
    
elif train_inputs == "fMRI/tfMRI.feat/reg_standard/stats/cope1.nii.gz":
    save_path = "data_analysis/cope1.pkl"
    
elif train_inputs == "fMRI/tfMRI.feat/reg_standard/stats/cope2.nii.gz":
    save_path = "data_analysis/cope2.pkl"
    
elif train_inputs == "fMRI/tfMRI.feat/reg_standard/stats/cope5.nii.gz":
    save_path = "data_analysis/cope5.pkl"

elif train_inputs == "dMRI/TBSS/stats/all_FA_skeletonised.nii.gz":
    save_path = "data_analysis/all_FA_skeletonised.pkl"
    
elif train_inputs == "dMRI/TBSS/stats/all_ICVF_skeletonised.nii.gz":
    save_path = "data_analysis/all_ICVF_skeletonised.pkl"
    
elif train_inputs == "dMRI/TBSS/stats/all_ISOVF_skeletonised.nii.gz":
    save_path = "data_analysis/all_ISOVF_skeletonised.pkl"
    
elif train_inputs == "dMRI/TBSS/stats/all_L1_skeletonised.nii.gz":
    save_path = "data_analysis/all_L1_skeletonised.pkl"
    
elif train_inputs == "dMRI/TBSS/stats/all_L2_skeletonised.nii.gz":
    save_path = "data_analysis/all_L2_skeletonised.pkl"
    
elif train_inputs == "dMRI/TBSS/stats/all_L3_skeletonised.nii.gz":
    save_path = "data_analysis/all_L3_skeletonised.pkl"
    
elif train_inputs == "dMRI/TBSS/stats/all_MD_skeletonised.nii.gz":
    save_path = "data_analysis/all_MD_skeletonised.pkl"
    
elif train_inputs == "dMRI/TBSS/stats/all_MO_skeletonised.nii.gz":
    save_path = "data_analysis/all_MO_skeletonised.pkl"
    
elif train_inputs == "dMRI/TBSS/stats/all_OD_skeletonised.nii.gz":
    save_path = "data_analysis/all_OD_skeletonised.pkl"
    
elif train_inputs == "dMRI/TBSS/stats/all_FA.nii.gz":
    save_path = "data_analysis/all_FA.pkl"
    
elif train_inputs == "dMRI/TBSS/stats/all_ICVF.nii.gz":
    save_path = "data_analysis/all_ICVF.pkl"
    
elif train_inputs == "dMRI/TBSS/stats/all_ISOVF.nii.gz":
    save_path = "data_analysis/all_ISOVF.pkl"
    
elif train_inputs == "dMRI/TBSS/stats/all_L1.nii.gz":
    save_path = "data_analysis/all_L1.pkl"
    
elif train_inputs == "dMRI/TBSS/stats/all_L2.nii.gz":
    save_path = "data_analysis/all_L2.pkl"
    
elif train_inputs == "dMRI/TBSS/stats/all_L3.nii.gz":
    save_path = "data_analysis/all_L3.pkl"
    
elif train_inputs == "dMRI/TBSS/stats/all_MD.nii.gz":
    save_path = "data_analysis/all_MD.pkl"
    
elif train_inputs == "dMRI/TBSS/stats/all_MO.nii.gz":
    save_path = "data_analysis/all_MO.pkl"
    
elif train_inputs == "dMRI/TBSS/stats/all_OD.nii.gz":
    save_path = "data_analysis/all_OD.pkl"
    
elif train_inputs == "T2_FLAIR/lesions/final_mask_to_MNI.nii.gz":
    save_path = "data_analysis/final_mask_to_MNI.pkl"
    
elif train_inputs == "T1/transforms/T1_to_MNI_warp_jac.nii.gz":
    save_path = "data_analysis/T1_to_MNI_warp_jac.pkl"
    
else:
    print("USE OTHER CODE!")


imaging_df = pd.DataFrame.from_dict(data_dict, orient="index", columns=dictionary_labels)
imaging_df.to_pickle(save_path)
print("Saved to: ", save_path)


In [ ]:
volume_index = 25

data_directory = "/well/win-biobank/projects/imaging/data/data3/subjectsAll/"
train_inputs = "fMRI/rfMRI_25.dr/dr_stage2.nii.gz"
len_subjects = len(subDirectoryList)

print(train_inputs)
data_dict = {}
dictionary_labels = ['min', 'max', 'mean', 'med', 'std', '1p', '25p', '75p', '99p', 'norm99p']
index = 0

for subject in subDirectoryList:
    index += 1
    path = os.path.join(os.path.expanduser("~"), data_directory, subject, train_inputs)
    volume = Image(path).data[:, :, :, volume_index]
    min_val, max_val, mean_val, med_val, std_val, perc1, perc25, perc75, perc99, norm99p = stats_calc(volume)
    data_dict[subject] = [min_val, max_val, mean_val, med_val, std_val, perc1, perc25, perc75, perc99, norm99p]
    del path,volume
    del min_val, max_val, mean_val, med_val, std_val, perc1, perc25, perc75, perc99, norm99p
    print("\r Processed {:.3f}%: {}/{} subjects".format(index/len_subjects * 100.0, index, len_subjects), end='')
    
save_path = 'data_analysis/dr_stage2' + '_' + str(volume_index) + '.pkl'
imaging_df = pd.DataFrame.from_dict(data_dict, orient="index", columns=dictionary_labels)
imaging_df.to_pickle(save_path)
print("Saved to: ", save_path)

In [ ]:
data_directory = "/well/win-biobank/projects/imaging/data/data3/subjectsAll/"
train_inputs = interesting_data_paths[0]
len_subjects = len(subDirectoryList)

print(train_inputs)

if train_inputs == "fMRI/rfMRI_25.dr/dr_stage2.nii.gz":
    data_dict = {}
    dictionary_labels = ['subject','index','min', 'max', 'mean', 'med', 'std', '1p', '25p', '75p', '99p', 'norm99p']
    index = 0
    for subject in subDirectoryList:
        index += 1
        path = os.path.join(os.path.expanduser("~"), data_directory, subject, train_inputs)
        volume = Image(path).data
        for i in range(25):
            sub_volume = volume[:, :, :, i]
            min_val, max_val, mean_val, med_val, std_val, perc1, perc25, perc75, perc99, norm99p = stats_calc(volume)
            key = str(subject)+'-'+str(i)
            data_dict[key] = [subject, i, min_val, max_val, mean_val, med_val, std_val, perc1, perc25, perc75, perc99, norm99p]
        del path,volume
        del min_val, max_val, mean_val, med_val, std_val, perc1, perc25, perc75, perc99, norm99p
        print("\r Processed {:.3f}%: {}/{} subjects".format(index/len_subjects * 100.0, index, len_subjects), end='')

    save_path = "data_analysis/dr_stage2.pkl"

    imaging_df = pd.DataFrame.from_dict(data_dict, orient="index", columns=dictionary_labels)
    imaging_df.to_pickle(save_path)
    print()
    print("Saved to: ", save_path)
else:
